In [ ]:
! pip install transformers
! pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.2 MB/s 
     |████████████████████████████████| 182 kB 71.1 MB/s 
     |████████████████████████████████| 7.6 MB 54.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 798 kB 5.2 MB/s 
     |████████████████████████████████| 529 kB 58.9 MB/s 
     |████████████████████████████████| 87 kB 8.7 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115940 sha256=ff8e9ebd943fb9176eb4bb2a566cb973d531eb503637b0ab6dcf177d70a62e65
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
Successfully built fire


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
from transformers import BertModel, BertTokenizer
import pytorch_lightning as pl
import torch.nn.functional as F
from torch import nn
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
#bert = BertModel.from_pretrained('bert-base-cased', output_attentions=True)

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/lsml2/amazon_review/train.csv').dropna()
test = pd.read_csv('/content/drive/MyDrive/lsml2/amazon_review/test.csv').dropna()

In [ ]:
train['class_index'] = train['class_index'].replace({2: 1, 1: 0})
test['class_index'] = test['class_index'].replace({2: 1, 1: 0})

In [ ]:
train, val = train_test_split(train, test_size=0.1)

In [ ]:
class AmazonDataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = df.class_index.to_list()
        self.texts = [tokenizer(text, 
                                padding='max_length', max_length = 64, truncation=True,
                                return_tensors="pt") for text in df['review_title']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        input_ids = batch_texts['input_ids']
        token_type_ids = batch_texts['token_type_ids']
        attention_mask = batch_texts['attention_mask']
        batch_y = self.get_batch_labels(idx)

        return input_ids, token_type_ids, attention_mask, batch_y

In [ ]:
train_dataloader = torch.utils.data.DataLoader(AmazonDataset(train), batch_size=32)
val_dataloader = torch.utils.data.DataLoader(AmazonDataset(val), batch_size=32)
test_dataloader = torch.utils.data.DataLoader(AmazonDataset(test), batch_size=32)

In [ ]:
class BertMNLIFinetuner(pl.LightningModule):

    def __init__(self):
        super(BertMNLIFinetuner, self).__init__()
        
        self.bert = BertModel.from_pretrained('bert-base-cased', output_attentions=True)
        
        self.W = nn.Linear(self.bert.config.hidden_size, 2)
        self.num_classes = 2


    def forward(self, input_ids, attention_mask, token_type_ids):
        
        input_ids = input_ids.squeeze(1)
        attention_mask = attention_mask.squeeze(1)
        token_type_ids = token_type_ids.squeeze(1)
          
        out = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        
        attn = out['attentions']
        h = out['last_hidden_state']
        
        h_cls = h[:, 0]
        logits = self.W(h_cls)
        return logits, attn

    def training_step(self, batch, batch_nb):

        input_ids, attention_mask, token_type_ids, label = batch
         
        y_hat, attn = self.forward(input_ids, attention_mask, token_type_ids)
        
        loss = F.cross_entropy(y_hat, label)
        
        tensorboard_logs = {'train_loss': loss}
        return {'loss': loss, 'log': tensorboard_logs}

    def validation_step(self, batch, batch_nb):

        input_ids, attention_mask, token_type_ids, label = batch
         
        y_hat, attn = self.forward(input_ids, attention_mask, token_type_ids)
        
        loss = F.cross_entropy(y_hat, label)
        
        a, y_hat = torch.max(y_hat, dim=1)
        val_acc = accuracy_score(y_hat.cpu(), label.cpu())
        val_acc = torch.tensor(val_acc)

        return {'val_loss': loss, 'val_acc': val_acc}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        avg_val_acc = torch.stack([x['val_acc'] for x in outputs]).mean()
        tensorboard_logs = {'val_loss': avg_loss, 'avg_val_acc': avg_val_acc}
        return {'avg_val_loss': avg_loss, 'progress_bar': tensorboard_logs}

    def test_step(self, batch, batch_nb):
        input_ids, attention_mask, token_type_ids, label = batch
        
        y_hat, attn = self.forward(input_ids, attention_mask, token_type_ids)
        
        a, y_hat = torch.max(y_hat, dim=1)
        test_acc = accuracy_score(y_hat.cpu(), label.cpu())
        self.log_dict({'test_acc': torch.tensor(test_acc)})
        return {'test_acc': torch.tensor(test_acc)}
        
    def test_epoch_end(self, outputs):

        avg_test_acc = torch.stack([x['test_acc'] for x in outputs]).mean()

        tensorboard_logs = {'avg_test_acc': avg_test_acc}
        self.log_dict(tensorboard_logs)
        return {'avg_test_acc': avg_test_acc, 'log': tensorboard_logs, 'progress_bar': tensorboard_logs}

    def configure_optimizers(self):
        return torch.optim.Adam([p for p in self.parameters() if p.requires_grad], lr=1e-05, eps=1e-08)


    def train_dataloader(self):
        return train_dataloader


    def val_dataloader(self):
        return val_dataloader


    def test_dataloader(self):
        return test_dataloader

In [ ]:
berttuner = BertMNLIFinetuner()

trainer = pl.Trainer(gpus=1, max_epochs=5, log_every_n_steps=10)    
trainer.fit(berttuner)

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


In [ ]:
trainer.test(berttuner)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      avg_test_acc          0.7556250095367432
        test_acc            0.7400000095367432
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_acc': 0.7400000095367432, 'avg_test_acc': 0.7556250095367432}]

In [ ]:
torch.save(berttuner, '/content/drive/MyDrive/lsml2/amazon_review/model.pt')